In [ ]:

from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType

In [ ]:


## run a spark session to load file
spark = SparkSession.builder.getOrCreate()
Schema = StructType().add("A", "string").add("rank", "double")
rank_df = spark.readStream.format('csv').option("delimiter",'\t').schema(Schema).load('hdfs:/p2t1.csv',header='true')

In [ ]:
# select rank>0.5 and output it as a csv file
output = rank_df.select(['A','rank']).where('rank>0.5')
query = output.coalesce(1).writeStream.format('csv').option("checkpointLocation", "hdfs:/").option("path",'gs://assignment2_p2/p2t1_whole.csv').start()

In [ ]:
spark.streams.active
query.awaitTermination()

In [ ]:
query.stop()

In [ ]:
# start another spark session to read file and count
spark2 = SparkSession.builder.getOrCreate()
rank_larger = spark2.read.format('csv').load('gs://assignment2_p2/p2t1_whole.csv')

In [ ]:
print(rank_larger.count())